In [ ]:
try:
    import opacus
    print("Opacus is already installed.")
except ModuleNotFoundError:
    print("Opacus not found. Installing...")
    !pip install opacus
    import opacus
    print("Opacus has been installed successfully.")

# try:
#     import peft
#     print("PEFT is already installed.")
# except ModuleNotFoundError:
#     print("PEFT not found. Installing...")
#     !pip install peft
#     import peft
#     print("PEFT has been installed successfully.")

try:
    import datasets
    print("Datasets is already installed.")
except ModuleNotFoundError:
    print("Datasets not found. Installing...")
    !pip install datasets
    import datasets
    print("Datasets has been installed successfully.")

Opacus not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 4.4 MB/s eta 0:00:00
Opacus has been installed successfully.
Datasets not found. Installing...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 22.6 MB/s eta 0:00:00
  Attemptin

In [ ]:
import pandas as pd
import numpy as np

splits = {'train': 'yelp_review_full/train-00000-of-00001.parquet', 'test': 'yelp_review_full/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["train"])
df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...


In [ ]:
df.isnull().sum()

,0
label,0
text,0


In [ ]:
df.dropna(inplace=True)

In [ ]:
df = df.iloc[:100001,:]

In [ ]:
df.shape

(100001, 2)

# Data Prep

In [ ]:
df_mod = df.copy()

In [ ]:
# converting text to lower case
df_mod = df_mod.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-8-20369b8d26b7>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_mod = df_mod.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
# removing contractions
contractions = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot", "couldn't": "could not", "didn't": "did not",
    "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did", "how'll": "how will",
    "how's": "how is", "I’d": "I would", "I'll": "I will", "I'm": "I am", "I've": "I have", "I’m": "I am",
    "isn't": "is not", "it'd": "it would", "it'll": "it will", "it's": "it is", "let's": "let us", "ma'am": "madam",
    "mightn't": "might not", "mustn't": "must not", "needn't": "need not", "shan't": "shall not", "she'd": "she would",
    "she'll": "she will", "she's": "she is", "shouldn't": "should not", "that'd": "that would", "that'll": "that will",
    "that's": "that is", "there'd": "there would", "there'll": "there will", "there's": "there is", "they'd": "they would",
    "they'll": "they will", "they're": "they are", "they've": "they have", "wasn't": "was not", "weren't": "were not",
    "what'll": "what will", "what's": "what is", "what've": "what have", "where'd": "where did", "where'll": "where will",
    "where's": "where is", "who'd": "who would", "who'll": "who will", "who's": "who is", "who've": "who have",
    "why'd": "why did", "why'll": "why will", "why's": "why is", "won't": "will not", "wouldn't": "would not",
}
def expand_contractions(text):
    # Check if the input is a float
    if isinstance(text, float):
        return text

    for contraction, expanded in contractions.items():
        text = text.replace(contraction, expanded)

    return text

# apply
df_mod['text'] = df_mod['text'].apply(expand_contractions)

In [ ]:
# removing puntuations
df_mod = df_mod.replace(r'[^\w\s]', '', regex=True)

In [ ]:
# stop words removal
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# for all col
df_mod['text'] = df_mod['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stop]) if x is not None else '')


In [ ]:
# stemming
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt_tab')
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def stem_text(text):
    if isinstance(text, str):
        words = nltk.word_tokenize(text)
        stemmed_words = [stemmer.stem(word) for word in words]
        return ' '.join(stemmed_words)


df_mod['text'] = df_mod['text'].apply(stem_text)

In [ ]:
df_mod.head(5)

,label,text
0,4,dr goldberg offer everyth look gener practitio...
1,1,unfortun frustrat dr goldberg patient repeat e...
2,3,go dr goldberg 10 year think one 1st patient s...
3,3,got letter mail last week said dr goldberg mov...
4,0,know dr goldberg like move arizona let tell st...


In [ ]:
df_mod.shape

(100001, 2)

# BERT

In [ ]:
pip install transformers datasets torch

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss

In [ ]:
# Encoding the target variable 'Popularity'
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])


In [ ]:
from sklearn.model_selection import train_test_split

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to tokenize the text data
def tokenize_texts(texts, max_length=256):
    return tokenizer.batch_encode_plus(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'  # Return PyTorch tensors
    )

# Tokenize the training and test data
train_encodings = tokenize_texts(X_train.tolist())
test_encodings = tokenize_texts(X_test.tolist())

torch.save(train_encodings, 'train_encodings.pt')
torch.save(test_encodings, 'test_encodings.pt')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_encodings = torch.load('train_encodings.pt', weights_only=False)
test_encodings = torch.load('test_encodings.pt', weights_only=False)

In [ ]:
y_train_tensor = torch.tensor(y_train.tolist(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.tolist(), dtype=torch.long)

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

In [ ]:
# Create Dataset objects
train_dataset = TextDataset(train_encodings, y_train_tensor)
test_dataset = TextDataset(test_encodings, y_test_tensor)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device))
    print(outputs.logits[:5])  # Check if all values are similar (over-regularized)
    break

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertConfig

class BERTForClassification(nn.Module):
    """
    BERT-based model for multi-class text classification.
    """

    def __init__(self, bert_model_name='bert-base-uncased', num_classes=5):
        super(BERTForClassification, self).__init__()

        # Initialize BERT model
        self.config = BertConfig.from_pretrained(
              bert_model_name,
              num_labels=num_classes,
          )
        self.bert = BertModel.from_pretrained(bert_model_name, config = self.config)

        # Define additional layers for classification
        self.dropout = nn.Dropout(p=0.3)  # Dropout layer to prevent overfitting
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, input_ids, attention_mask):
        # Get the pooled output from BERT
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output

        # Classification head
        x = self.dropout(bert_output)
        x = self.relu1(self.fc1(x))
        x = self.dropout(x)
        x = self.relu2(self.fc2(x))
        x = self.dropout(x)
        logits = self.fc3(x)

        return logits

In [ ]:
import torch.optim as optim
from sklearn.metrics import f1_score, confusion_matrix
from tqdm import tqdm  # For progress display
from opacus.optimizers import DPOptimizer
from opacus import PrivacyEngine
from opacus.grad_sample import GradSampleModule
from opacus.validators import ModuleValidator

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model

model = BERTForClassification().to(device)
# model = GradSampleModule(model)

trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.fc1, model.fc2, model.fc3]
total_params = 0
trainable_params = 0

for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for layer in trainable_layers:
    for p in layer.parameters():
        p.requires_grad = True
        trainable_params += p.numel()

print(f"Total parameters count: {total_params:,}") # ~108M
print(f"Trainable parameters count: {trainable_params:,}")


errors = ModuleValidator.validate(model, strict=False)
errors[-5:]

Total parameters count: 109,589,253
Trainable parameters count: 7,678,789


[]

In [ ]:
# Optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-5,
    weight_decay=1e-6)
# base_optimizer = AdamW(
#     model.parameters(),
#     lr=5e-5,
#     weight_decay=0.01
# )

# # 4. Wrap it with DPOptimizer
# optimizer = DPOptimizer(
#     optimizer=base_optimizer,
#     noise_multiplier=1.0,
#     max_grad_norm=1.0,
#     expected_batch_size=128  #CRUCIAL: Set this to your actual batch size
# )
criterion = nn.CrossEntropyLoss()
target_epsilon = 5.0
delta = 1/len(train_loader)

privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    max_grad_norm = 1.0,
    epochs=3,
    target_epsilon=target_epsilon,
    target_delta=delta,
    grad_sample_mode="ghost"
)

/usr/local/lib/python3.11/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


In [ ]:
# Optional: Learning rate scheduler
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Training loop

from opacus.utils.batch_memory_manager import BatchMemoryManager
num_epochs = 1  # Adjust as needed
MAX_PHYSICAL_BATCH_SIZE = 16

# for epoch in range(num_epochs):
model.train()
total_loss = 0
with BatchMemoryManager(
        data_loader=train_loader,
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
        optimizer=optimizer
    ) as memory_safe_data_loader:
      for step, batch in enumerate(tqdm(memory_safe_data_loader)):
          print("Input IDs shape:", batch['input_ids'].shape)
          print("Attention Mask shape:", batch['attention_mask'].shape)
          print("Labels shape:", batch['labels'].shape)
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          optimizer.zero_grad()  # Reset gradients

          # Forward pass
          outputs = model(input_ids, attention_mask)
          loss = criterion(outputs, labels)

          # Backward pass
          loss.backward()
          optimizer.step()

          total_loss += loss.item()
          if (step + 1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(delta)
                print(
                    f"(ε = {epsilon:.2f}, δ = {delta})"
                )

      # Optional: Step the scheduler
      # scheduler.step()

      avg_train_loss = total_loss / len(train_loader)
      print(f"Training Loss: {avg_train_loss:.4f}")





  0%|          | 0/5000 [00:00<?, ?it/s]

Input IDs shape: torch.Size([15, 256])
Attention Mask shape: torch.Size([15, 256])
Labels shape: torch.Size([15])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
  0%|          | 1/5000 [00:01<2:00:42,  1.45s/it]

Input IDs shape: torch.Size([15, 256])
Attention Mask shape: torch.Size([15, 256])
Labels shape: torch.Size([15])


  0%|          | 2/5000 [00:02<2:01:08,  1.45s/it]

...

Input IDs shape: torch.Size([15, 256])
Attention Mask shape: torch.Size([15, 256])
Labels shape: torch.Size([15])


5324it [1:19:34,  1.34it/s]

Input IDs shape: torch.Size([15, 256])
Attention Mask shape: torch.Size([15, 256])
Labels shape: torch.Size([15])


5325it [1:19:35,  1.12it/s]

Training Loss: 13.7124


In [ ]:
# Evaluation loop
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, dim=1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to NumPy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Calculate F1-score
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"F1 Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", conf_matrix)

# Save model, optionally only if it has improved the best validation score
torch.save(model.state_dict(), 'bert_classification_model.pth')

 13%|█▎        | 20/157 [02:56<20:02,  8.77s/it]